In [1]:
import pandas as pd
from threading import Thread
from queue import Queue
import multiprocessing

In [2]:
xcelLocation = "../xcels/"
!export xcelLocation="../xcels/"
csvFileName = "MarketWatchPlus-1398_5_7"
csvFileName = "1382-2-2"

In [3]:
!ls $xcelLocation | grep ".csv" | grep 13 > csvFiles
tmp = !cat csvFiles
csvFiles = [name[:-4] for name in tmp]

In [4]:
def readCsv(xcelLocation, csvFileName):
    df=pd.read_csv(xcelLocation + csvFileName + '.csv', sep=',',header=[0])
    return df

def addDiff(df): 
    diffTable = df.loc[:,["name", "min", "max"]]
    sumdif,count = (0,0)
    difs = []
    for row in range(len(diffTable)):
        minim = diffTable.at[row, "min"]
        maxim = diffTable.at[row, "max"]
        if(minim != 0):
            count += 1
            sumdif += (maxim - minim)/minim
            difs.append((maxim - minim)/minim * 100)
        else:
            difs.append(0)
    diffTable.loc[:,"diff"] = difs
    df.loc[:,"diff"] = difs
    return df, diffTable, sumdif, count

In [5]:
df = readCsv(xcelLocation=xcelLocation, csvFileName=csvFileName)
df, diffTable, sumdif, count = addDiff(df)

In [6]:
def readThread(threadname, q, qsum, qcount):
    while not q.empty():
        csvNames = q.get()
        q.task_done()
        
        for csvName in csvNames:
            df = readCsv(xcelLocation=xcelLocation, csvFileName=csvName)
            df, diffTable, sumdif, count = addDiff(df)
            qsum.put(sumdif)
            qcount.put(count)
    print(str(threadname) + " done")

In [7]:
# print(sumdif/count)
# diffTable
sums, counts = 0,0
batchSize = 10
i = 0
numThread = 8
workers = []

pool = multiprocessing.Pool(processes=numThread)
m = multiprocessing.Manager()
queue = m.Queue()
qsum = m.Queue()
qcount = m.Queue()


while i*batchSize < len(csvFiles):
    if (i+1)*batchSize < len(csvFiles):
        queue.put(csvFiles[i*batchSize:(i+1)*batchSize])
    else:
        queue.put(csvFiles[i*10:])
    i+=1
print(len(csvFiles))
print(queue.qsize())

for i in range(numThread):
#     workers.append(Thread(target=readThread, args=("Thread-" + str(i), queue, qsum, qcount)))
#     workers.append(pool.apply_async(readThread, ("Thread-" + str(i), queue, qsum, qcount,)))
    workers.append(multiprocessing.Process(target=readThread, args=("Thread-" + str(i),
                                                                    queue, qsum, qcount)))
    workers[i].start()

for i in range(numThread):
    workers[i].join()


4228
423
Thread-2 done
Thread-1 done
Thread-6 done
Thread-5 done
Thread-3 done
Thread-4 done
Thread-7 done
Thread-0 done


In [8]:
sums = []
counts = []
while not qsum.empty():
    sums.append(qsum.get())
while not qcount.empty():
    counts.append(qcount.get())

In [9]:
sumss, countss = 0,0
for num in sums:
    sumss += num
for num in counts:
    countss += num
print(sumss/countss)

0.030881421186892404


In [10]:

# with open('7mivePrice.csv', 'a', encoding="utf-16") as csvfile:
#     excel = csv.writer(csvfile)
#     print("test" + str(name[0]) + " " + str(price[0]))
#     for a, b, c, d in zip(price, name, moredis, weight):
#         excel.writerow([str(a), str(b), now, str(c), str(d)])
         #excel.writerow([str(a).encode(sys.stdout.encoding, errors='replace')] + [str(b).encode(sys.stdout.encoding, errors='replace')])


In [11]:
xcel57 = readCsv(xcelLocation=xcelLocation, csvFileName="1398-5-7")
xcel58 = readCsv(xcelLocation=xcelLocation, csvFileName="1398-5-8")
xcel59 = readCsv(xcelLocation=xcelLocation, csvFileName="1398-5-9")
_,xcel57,_,_ = addDiff(xcel57)
_,xcel58,_,_ = addDiff(xcel58)
_,xcel59,_,_ = addDiff(xcel59)

In [12]:
sumdiffList = []
for index in range(len(xcel57)):
    sumdiff = 0
    name = xcel57.loc[index,"name"]
    row = xcel57.loc[xcel57["name"] == name]
    sumdiff += row.loc[row.index[0], "diff"]
    row = xcel58.loc[xcel58["name"] == name]
    if len(row) > 0:
        sumdiff += row.loc[row.index[0], "diff"]
    row = xcel59.loc[xcel59["name"] == name]
    if len(row) > 0:
        sumdiff += row.loc[row.index[0], "diff"]
    sumdiffList.append((sumdiff, name))

In [13]:
sumdiffList.sort(reverse=True)

In [14]:
sumdiffList

[(130.88225472517198, 'اختيارخ كچاد-5800-1398/07/03'),
 (107.77027027027026, 'اختيارخ وتجارت-500-1398/07/07'),
 (66.23376623376623, 'اختيارخ وتجارت-600-1398/07/07'),
 (65.5, 'اختيارخ شتران-9200-1398/08/12'),
 (53.0254106415836, 'اختيارخ وغدير-2600-1398/08/22'),
 (47.3067834215852, 'پشم\u200c بافي\u200c توس'),
 (44.98024804777216, 'اختيارخ فملي -4850-1398/08/19'),
 (42.12884816955107, 'فولاد افزا سپاهان'),
 (40.64875397450599, 'رهشاد سپاهان (سهامي عام'),
 (39.34971861600497, 'اقتصادي و خودكفايي آزادگان'),
 (38.575581967360705, 'س. ساختماني نظام مهندسي ايران'),
 (38.15396137820281, 'بين المللي ساروج بوشهر'),
 (37.574160723615464, 'توليد مواداوليه الياف مصنوعي'),
 (35.38553369963799, 'ح . خدمات فني فولاد يزد'),
 (35.23858544166568, 'شركت ليزينگ آريا دانا'),
 (35.12893642886165, 'پلي اكريل ايران'),
 (33.61002354403629, 'نيرو سرمايه'),
 (33.33333333333333, 'اختيارخ فولاد-4500-1398/08/26'),
 (32.39715840231041, 'لوازم\u200c خانگي\u200c پارس\u200c'),
 (32.05246548323471, 'تجهيزنيروي\u200cزنگا